In [1]:
pip install facenet_pytorch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 12.9 MB/s eta 0:00:00


In [2]:
from facenet_pytorch import MTCNN, InceptionResnetV1
import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from PIL import Image
from torch import nn

import random
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from torch.utils.data import Dataset
from torchvision import transforms
import glob
import os
# from tensorflow.keras.models import load_model

# from tensorflow.keras.models import Model
import os
import warnings


from torch.utils.data import random_split
from torch.utils.data import DataLoader,Dataset

In [3]:
class CustomDataset(Dataset):
    def __init__(self, csv_file, transform=None):
        self.data = pd.read_csv(csv_file)
        self.transform = transform
        self.gender_mapping = {'male': 0, 'female': 1}
        self.ethnicity_mapping = {'white': 0, 'black': 1, 'asian': 2, 'hispanic': 3}
        self.data['nameNum'] = self.data['name'].astype('category').cat.codes
        self.data['nameNum'] = self.data['nameNum'].astype(int)


    def __len__(self):
        return len(self.data)



    def getAgeLabel(self,value1):

    # Define class ranges
        class_ranges = [(15, 22),(22,40),(40,60),(60,80)]

    # Check if both values fall into the same class range
        if(class_ranges[0][0]<=value1 and value1<class_ranges[0][1]):
            return 0
        elif(class_ranges[1][0]<=value1 and value1<class_ranges[1][1]):
            return 1
        elif(class_ranges[2][0]<=value1 and value1<class_ranges[2][1]):
            return 2
        elif(class_ranges[3][0]<=value1 and value1<class_ranges[3][1]):
            return 3
        else:
            return 0


    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        image_path = '/content/drive/MyDrive/AI/archive/CELEBTEST/CELEBTEST/'+row['name']+'/' + row['filename']  # Assuming images are in a folder named 'images'

        try:
            image = Image.open(image_path)
        except Exception as e:
            # Handle the error, for example, you can return a placeholder image
            print("here")
            #self.__getitem__(idx + 1)
            #image = Image.new('RGB', (224, 224))  # Create a blank image

        image = Image.open(image_path)
        age = row['age']

        if(row['age']<=0):
            age=35
        label = {
            'age': self.getAgeLabel(age),
            'gender': self.gender_mapping.get(row['gender'], 0),  # -1 for unknown
            'ethnicity': self.ethnicity_mapping.get(row['ethnicity'], 0),
            'age1':age,
            'name': row['nameNum']

        }
        #print(row['name'])
        #change to numbers
        if self.transform:
            image = self.transform(image)

        return image, label


In [4]:
transform = transforms.Compose([
    transforms.Resize((112, 112)),  # Resize the image to the desired size
    transforms.ToTensor(),          # Convert the image to a tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize the image
])


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
trainSet = CustomDataset('/content/drive/MyDrive/AI/archive/celebSET_final_v1.csv', transform=transform)
trainloader = DataLoader(trainSet, batch_size=256, shuffle=True, num_workers = 2)

testSet = CustomDataset('/content/drive/MyDrive/AI/archive/final_clebSET_test.csv', transform=transform)
testloader = DataLoader(testSet, batch_size=2, shuffle=False)

In [7]:
import torch
import torch.nn as nn
from torch.nn import Linear, Conv2d, BatchNorm1d, BatchNorm2d, PReLU, ReLU, Sigmoid, Dropout, MaxPool2d, \
    AdaptiveAvgPool2d, Sequential, Module
from collections import namedtuple


# Support: ['IR_50', 'IR_101', 'IR_152', 'IR_SE_50', 'IR_SE_101', 'IR_SE_152']


class Flatten(Module):
    def forward(self, input):
        return input.view(input.size(0), -1)


def l2_norm(input, axis=1):
    norm = torch.norm(input, 2, axis, True)
    output = torch.div(input, norm)

    return output


class SEModule(Module):
    def __init__(self, channels, reduction):
        super(SEModule, self).__init__()
        self.avg_pool = AdaptiveAvgPool2d(1)
        self.fc1 = Conv2d(
            channels, channels // reduction, kernel_size=1, padding=0, bias=False)

        nn.init.xavier_uniform_(self.fc1.weight.data)

        self.relu = ReLU(inplace=True)
        self.fc2 = Conv2d(
            channels // reduction, channels, kernel_size=1, padding=0, bias=False)

        self.sigmoid = Sigmoid()

    def forward(self, x):
        module_input = x
        x = self.avg_pool(x)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.sigmoid(x)

        return module_input * x


class bottleneck_IR(Module):
    def __init__(self, in_channel, depth, stride):
        super(bottleneck_IR, self).__init__()
        if in_channel == depth:
            self.shortcut_layer = MaxPool2d(1, stride)
        else:
            self.shortcut_layer = Sequential(
                Conv2d(in_channel, depth, (1, 1), stride, bias=False), BatchNorm2d(depth))
        self.res_layer = Sequential(
            BatchNorm2d(in_channel),
            Conv2d(in_channel, depth, (3, 3), (1, 1), 1, bias=False), PReLU(depth),
            Conv2d(depth, depth, (3, 3), stride, 1, bias=False), BatchNorm2d(depth))

    def forward(self, x):
        shortcut = self.shortcut_layer(x)
        res = self.res_layer(x)

        return res + shortcut


class bottleneck_IR_SE(Module):
    def __init__(self, in_channel, depth, stride):
        super(bottleneck_IR_SE, self).__init__()
        if in_channel == depth:
            self.shortcut_layer = MaxPool2d(1, stride)
        else:
            self.shortcut_layer = Sequential(
                Conv2d(in_channel, depth, (1, 1), stride, bias=False),
                BatchNorm2d(depth))
        self.res_layer = Sequential(
            BatchNorm2d(in_channel),
            Conv2d(in_channel, depth, (3, 3), (1, 1), 1, bias=False),
            PReLU(depth),
            Conv2d(depth, depth, (3, 3), stride, 1, bias=False),
            BatchNorm2d(depth),
            SEModule(depth, 16)
        )

    def forward(self, x):
        shortcut = self.shortcut_layer(x)
        res = self.res_layer(x)

        return res + shortcut


class Bottleneck(namedtuple('Block', ['in_channel', 'depth', 'stride'])):
    '''A named tuple describing a ResNet block.'''


def get_block(in_channel, depth, num_units, stride=2):

    return [Bottleneck(in_channel, depth, stride)] + [Bottleneck(depth, depth, 1) for i in range(num_units - 1)]


def get_blocks(num_layers):
    if num_layers == 50:
        blocks = [
            get_block(in_channel=64, depth=64, num_units=3),
            get_block(in_channel=64, depth=128, num_units=4),
            get_block(in_channel=128, depth=256, num_units=14),
            get_block(in_channel=256, depth=512, num_units=3)
        ]
    elif num_layers == 100:
        blocks = [
            get_block(in_channel=64, depth=64, num_units=3),
            get_block(in_channel=64, depth=128, num_units=13),
            get_block(in_channel=128, depth=256, num_units=30),
            get_block(in_channel=256, depth=512, num_units=3)
        ]
    elif num_layers == 152:
        blocks = [
            get_block(in_channel=64, depth=64, num_units=3),
            get_block(in_channel=64, depth=128, num_units=8),
            get_block(in_channel=128, depth=256, num_units=36),
            get_block(in_channel=256, depth=512, num_units=3)
        ]

    return blocks


class Backbone(Module):
    def __init__(self, input_size, num_layers, mode='ir'):
        super(Backbone, self).__init__()
        assert input_size[0] in [112, 224], "input_size should be [112, 112] or [224, 224]"
        assert num_layers in [50, 100, 152], "num_layers should be 50, 100 or 152"
        assert mode in ['ir', 'ir_se'], "mode should be ir or ir_se"
        blocks = get_blocks(num_layers)
        if mode == 'ir':
            unit_module = bottleneck_IR
        elif mode == 'ir_se':
            unit_module = bottleneck_IR_SE
        self.input_layer = Sequential(Conv2d(3, 64, (3, 3), 1, 1, bias=False),
                                      BatchNorm2d(64),
                                      PReLU(64))
        if input_size[0] == 112:
            self.output_layer = Sequential(BatchNorm2d(512),
                                           Dropout(),
                                           Flatten(),
                                           Linear(512 * 7 * 7, 512),
                                           BatchNorm1d(512))
        else:
            self.output_layer = Sequential(BatchNorm2d(512),
                                           Dropout(),
                                           Flatten(),
                                           Linear(512 * 14 * 14, 512),
                                           BatchNorm1d(512))

        modules = []
        for block in blocks:
            for bottleneck in block:
                modules.append(
                    unit_module(bottleneck.in_channel,
                                bottleneck.depth,
                                bottleneck.stride))
        self.body = Sequential(*modules)

        self._initialize_weights()

    def forward(self, x):
        x = self.input_layer(x)
        x = self.body(x)
        x = self.output_layer(x)

        return x

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.xavier_uniform_(m.weight.data)
                if m.bias is not None:
                    m.bias.data.zero_()
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()
            elif isinstance(m, nn.BatchNorm1d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()
            elif isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight.data)
                if m.bias is not None:
                    m.bias.data.zero_()


def IR_50(input_size):
    """Constructs a ir-50 model.
    """
    model = Backbone(input_size, 50, 'ir')

    return model


def IR_101(input_size):
    """Constructs a ir-101 model.
    """
    model = Backbone(input_size, 100, 'ir')

    return model


def IR_152(input_size):
    """Constructs a ir-152 model.
    """
    model = Backbone(input_size, 152, 'ir')

    return model


def IR_SE_50(input_size):
    """Constructs a ir_se-50 model.
    """
    model = Backbone(input_size, 50, 'ir_se')

    return model


def IR_SE_101(input_size):
    """Constructs a ir_se-101 model.
    """
    model = Backbone(input_size, 100, 'ir_se')

    return model


def IR_SE_152(input_size):
    """Constructs a ir_se-152 model.
    """
    model = Backbone(input_size, 152, 'ir_se')

    return model

In [8]:
arcFaceModel = IR_50([112,112])
arcFaceModel.load_state_dict(torch.load("/content/drive/MyDrive/AI/backbone_ir50_ms1m_epoch120.pth", map_location=torch.device('cpu')))

<All keys matched successfully>

In [9]:
warnings.filterwarnings("ignore", category=DeprecationWarning)  # Ignore DeprecationWarnings
warnings.filterwarnings("ignore", category=FutureWarning)
#os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

print('Running on device: {}'.format(device))

Running on device: cuda:0


In [10]:
warnings.filterwarnings("ignore", category=UserWarning)


processedTensorTrain = torch.zeros((len(testSet), 512))
#ageLabelTensor = torch.zeros((len(testSet)))
#genderLabelTensorTrain = torch.zeros((len(testSet)))
#ethnLabelTensor = torch.zeros((len(testSet)))
identityLabel = torch.zeros(len(testSet))
arcFaceModel.to(device)
# sensitivity = 1
# epsilon = 0.1
count = 0
my_image=[]

for i,data in enumerate(testloader):

    inputs = data[0].to(device=device)
    age_label = data[1]['age'].to(device=device)
    gender_label = data[1]['gender'].to(device=device)
    ethn_label = data[1]['ethnicity'].to(device=device)
    ident_label = data[1]['name'].to(device=device)
    #inputs = inputs[:, [2, 1, 0], :, :]
    print(inputs.shape)
    embeddings = arcFaceModel(inputs)
    for j in range(0,embeddings.shape[0]):

       if ident_label[j].item()=="murali":

        my_image.append(count+j)

    processedTensorTrain[count : count + embeddings.shape[0]] = torch.tensor(embeddings)

    #ageLabelTensor[count:count+age_label.shape[0]] = age_label
    #genderLabelTensorTrain[count:count+gender_label.shape[0]] = gender_label
    #ethnLabelTensor[count:count+ethn_label.shape[0]] = ethn_label
    identityLabel[count:count+ident_label.shape[0]] = ident_label


    count = count + embeddings.shape[0]

    print(count)

torch.Size([2, 3, 112, 112])
2
torch.Size([2, 3, 112, 112])
4
torch.Size([2, 3, 112, 112])
6
torch.Size([2, 3, 112, 112])
8
torch.Size([2, 3, 112, 112])
10
torch.Size([2, 3, 112, 112])
12
torch.Size([2, 3, 112, 112])
14
torch.Size([2, 3, 112, 112])
16
torch.Size([2, 3, 112, 112])
18
torch.Size([2, 3, 112, 112])
20
torch.Size([2, 3, 112, 112])
22
torch.Size([2, 3, 112, 112])
24
torch.Size([2, 3, 112, 112])
26
torch.Size([2, 3, 112, 112])
28
torch.Size([2, 3, 112, 112])
30
torch.Size([2, 3, 112, 112])
32
torch.Size([2, 3, 112, 112])
34
torch.Size([2, 3, 112, 112])
36
torch.Size([2, 3, 112, 112])
38
torch.Size([2, 3, 112, 112])
40
torch.Size([2, 3, 112, 112])
42
torch.Size([2, 3, 112, 112])
44
torch.Size([2, 3, 112, 112])
46
torch.Size([2, 3, 112, 112])
48
torch.Size([2, 3, 112, 112])
50
torch.Size([2, 3, 112, 112])
52
torch.Size([2, 3, 112, 112])
54
torch.Size([2, 3, 112, 112])
56
torch.Size([2, 3, 112, 112])
58
torch.Size([2, 3, 112, 112])
60
torch.Size([2, 3, 112, 112])
62
torch.Size([

In [12]:
identityAccuracy = 0

count = 0

while(count < processedTensorTrain.shape[0]):

    currentFace = processedTensorTrain[count : count + 1]
    cosine_similarity = nn.functional.cosine_similarity(currentFace, processedTensorTrain)
    if(identityLabel[torch.topk(cosine_similarity, k = 2)[1][1]].item() == identityLabel[count].item()):
        identityAccuracy = identityAccuracy +1
    count = count + 1
    # print(count)
print("Face Verification Accuracy: ",identityAccuracy/count)

Face Verification Accuracy:  0.353125


As per the paper future work is about exploring how to make the face recognition model not invertible so that face images cannot be easily reconstructed from model weights to protect privacy.Instead of storing raw face images, extract relevant features and hash them. Hash functions are designed to be one-way, making it difficult to reverse the process and reconstruct the original input.:Proposed Algorithm:
Adding Hashing Layer:

Introduced a cryptographic hash function as a layer in the model after obtaining the embeddings.
The hash layer will transform the embeddings into hash codes using a secure cryptographic hash function.Adding a Hash Layer
Define the Hash Layer Class:

Create a new class named HashLayer that extends torch.nn.Module.
In the constructor (__init__), specify the input size, which is the dimensionality of the embeddings.
Initialization:

Initialize the HashLayer with the desired input size.
Forward Pass:

In the forward pass (forward method), apply a cryptographic hash function to each embedding in the input.
Convert the hashed values into a tensor and return.

In [13]:
import torch
import torch.nn as nn
from torch.nn import Linear, Conv2d, BatchNorm1d, BatchNorm2d, PReLU, ReLU, Sigmoid, Dropout, MaxPool2d, \
    AdaptiveAvgPool2d, Sequential, Module
from collections import namedtuple
import hashlib

class HashLayer(nn.Module):
    def __init__(self, input_size):
        super(HashLayer, self).__init__()
        self.input_size = input_size

    def forward(self, x):
        # Converting the embeddings to a tensor
        x_tensor = x.view(x.size(0), -1)

        # Convert the tensor to a numpy array and then to bytes
        x_bytes = x_tensor.cpu().detach().numpy().tobytes()

        # Apply SHA-256 hash
        hashed_embeddings = hashlib.sha256(x_bytes).digest()

        return torch.tensor(list(hashed_embeddings))

class Backbone(nn.Module):
    def __init__(self, input_size, num_layers, mode='ir'):
        super(Backbone, self).__init__()

        # Instantiate HashLayer
        self.hash_layer = HashLayer(input_size)

    def forward(self, x):
        x = self.input_layer(x)
        x = self.body(x)
        x = self.output_layer(x)

        # Apply HashLayer for privacy protection
        hashed_embeddings = self.hash_layer(x)

        return hashed_embeddings



After Introducing HashLayer after forward layer we can achieve privacy and protection to the input images .Face Recognition Model with Privacy Protection Algorithm:
Input:

Face images dataset with corresponding labels (age, gender, ethnicity, etc.).
Preprocessing:

Load the face recognition model architecture (e.g., IR_50).
Load pre-trained weights for the model.
Instantiate the model and move it to the device (GPU or CPU).
Define HashLayer Class:

Define a HashLayer class that inherits from nn.Module.
The HashLayer class should have a constructor to store the input size (embedding dimension).
Implement the forward method to apply a cryptographic hash function (e.g., SHA-256) to each embedding in a tensor.
Modify Backbone Class:

Modify the existing Backbone class to include the HashLayer after the original layers.
Instantiate an instance of the HashLayer in the constructor of the Backbone class.
Model Training (if applicable):

If training is required, define a loss function and an optimizer.
Train the model on the face image dataset, considering the hash layer as part of the training process.
Privacy Protection during Inference:

During face recognition inference:
Input a face image into the modified face recognition model.
The image passes through the original layers (input_layer, body, output_layer).
After the original layers, the embeddings pass through the HashLayer for privacy protection.
The output of the model is the hashed embeddings.
Face Verification:

To verify the identity of a face:
Compute the cosine similarity between the hashed embeddings of the target face and all other faces in the dataset.
If the top-k most similar faces include the correct identity, consider it a successful verification.
Output:

The output of the model during inference is a hashed representation of the face embeddings.
Privacy is protected by the one-way nature of the hash function